# Legislación

## Contexto
Los cerros que rodean la planicie central de la cuenca de Santiago imponen fuertes restricciones a la circulación de vientos y, por ende, a la renovación del aire al interior de la cuenca. Por ello, en épocas de estabilidad atmosférica los contaminantes quedan atrapados dentro de la cuenca que alberga a la ciudad de Santiago.

Por esta situación en particular santiago se ha visto varias veces enfrentado con una contaminación muy alta, llegando a ser declarada zona saturada por Ozono, Material Particulado, Monoxido de Carbono y Dioxido de Nitrógeno en 1996, el año anterior se ordena la creación del _Plan de Prevención y Descontaminación Atmosférica_ de la Región Metropolitana, que desde 1998 norma con el objetivo de:

El PPDA tiene por objetivo cumplir con las normas de calidad de aire cuyo propósito principal es la protección de la salud de la población. Ésta se encuentra habitualmente expuesta a concentraciones de contaminantes que afectan su bienestar físico y su calidad de vida en general. -- DS N°16 de 1998, Minsegpres, CONAMA

Con esto se realizan desde el mismo año mediciones diarias de distintos agentes contaminantes para evaluar tanto la situación actual del aire en santiago como para medir el rendimiento del PPDA.

| Fecha        | Tipo    | Nombre                                 |
|--------------|---------|----------------------------------------|
| 26-oct-95    | Decreto | Procedimiento PPDA                     |
| 01-ago-96    | Decreto | Zona saturada por O3 MP10 MP2,5 CO NO2 |
| 06-jun-98    | Decreto | Establece PPDA                         |
| 29-ene-04    | Decreto | Actualiza PPDA                         |
| 16-abr-10    | Decreto | Actualiza PPDA                         |
| 15-nov-14    | Decreto | Zona Saturada MP2,5                    |
| 24-nov-17    | Decreto | Establece PPDA                         |


Aquí intentaremos responder si estas legislaciones y regulaciones tienen algún efecto real sobre la disminución de la contaminación en la cuenca de Santiago.

In [8]:
# Imports used on the script

import matplotlib
matplotlib.use('tkagg')

import matplotlib.pyplot as plt
import numpy as np
from sklearn import linear_model
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
import pandas as pd

In [9]:
model = Pipeline([('poly', PolynomialFeatures(degree=3)),
                  ('linear', LinearRegression(fit_intercept=False))])
# fit to an order-3 polynomial data
x = np.arange(5)
y = 3 - 2 * x + x ** 2 - x ** 3
model = model.fit(x[:, np.newaxis], y)
model.named_steps['linear'].coef_

array([ 3., -2.,  1., -1.])

In [38]:
# Import climate data
df = pd.read_csv("C:/Users/basty/Documents/intro_mineria_dcc_2019/Proyecto/ClimaYEmisionesData/DatosHito1Clean.csv")
df[:10]

,Fecha,anno,mes,dia,O3,MP10,CO,SO2,MP25,NO2,Tmedia,Tmax,Precipitaciones
0,970402,1997,4,2,18.8750,111.565,NaN,NaN,NaN,NaN,17.6,27.5,0.0
1,970403,1997,4,3,14.2083,141.333,NaN,NaN,NaN,NaN,16.0,27.2,0.0
2,970404,1997,4,4,30.4545,156.950,NaN,NaN,NaN,NaN,17.6,31.1,0.0
3,970405,1997,4,5,NaN,0.000,NaN,NaN,NaN,NaN,16.9,29.6,0.0
4,970406,1997,4,6,NaN,0.000,NaN,NaN,NaN,NaN,15.2,25.6,0.0
5,970407,1997,4,7,NaN,0.000,NaN,NaN,NaN,NaN,16.0,27.0,0.0
6,970408,1997,4,8,20.0000,131.292,1.18750,NaN,NaN,NaN,14.8,23.3,0.0
7,970409,1997,4,9,33.1666,176.000,2.22500,NaN,NaN,NaN,17.1,28.7,0.0
8,970410,1997,4,10,31.1667,198.333,3.37083,NaN,NaN,NaN,18.0,31.8,0.0
9,970411,1997,4,11,27.5833,162.042,1.98333,NaN,NaN,NaN,16.4,29.0,0.0


In [39]:
df[(df['anno']!=1998) & (df['anno']==2004)]

,Fecha,anno,mes,dia,O3,MP10,CO,SO2,MP25,NO2,Tmedia,Tmax,Precipitaciones
2465,40101,2004,1,1,29.7083,55.5000,0.466666,3.82083,24.7083,NaN,20.9,30.4,0.0
2466,40102,2004,1,2,28.1667,48.6250,0.170833,3.93000,19.6250,NaN,21.7,32.2,0.0
2467,40103,2004,1,3,27.8333,49.0833,0.141666,4.91250,20.3333,NaN,22.0,31.8,0.0
2468,40104,2004,1,4,24.8750,44.3333,0.116666,3.49333,18.4167,NaN,20.6,30.4,0.0
2469,40105,2004,1,5,25.6250,46.5416,0.133333,5.02166,19.5000,NaN,21.7,31.3,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2826,41227,2004,12,27,21.5833,39.6250,0.120833,8.73333,25.1250,NaN,19.0,25.8,0.0
2827,41228,2004,12,28,19.9583,44.9583,0.204166,16.12270,26.6250,NaN,19.8,26.4,0.0
2828,41229,2004,12,29,28.8333,46.2916,0.200000,7.36094,30.0417,NaN,21.2,31.0,0.0
2829,41230,2004,12,30,21.5833,39.5833,0.320833,8.20173,19.8750,NaN,21.3,31.1,0.0
